In [ ]:
from ambientperiod.builder.build_period import BuildPeriod


In [ ]:
import matplotlib.pyplot as plt

# Set the font family and size
plt.rcParams['font.family'] = 'arial'
plt.rcParams['font.size'] = 10 
plt.rcParams['font.stretch'] = 'condensed'

In [ ]:
# Configuration (defined here instead of a config.py)
config = {
    "Fs": 244,                          # % Frecuencia de muestreo
    "vent": 30,                         # % Duracion de las ventanas
    "STA": 1,       "LTA": 30,          # % Algoritmo STA/LTA
    "vmin": 0.1,    "vmax": 2.5,        # % Limites algoritmo STA/LTA
    "p": 0.05,                          # % R para tapper de ventana
    "f1": 1.0,      "f2": 50.0,         # % Límites de ancho de banda de frecuencia     
    "bexp": 60                          # % Constante de suavizado (Konno & Ohmachi 1998)
}

In [ ]:
from ambientperiod.analysis.sua_vent import show_smoothing_example

show_smoothing_example()


In [ ]:
# signal_path = r'C:\Users\ppala\OneDrive\01. Brain\11. GitHub\AmbientSoilPeriod\signals\TS01.txt'
# builder = BuildPeriod(signal_path, config , display_figures=False)

In [ ]:
signal_path = r'C:\Users\ppala\OneDrive\01. Brain\11. GitHub\AmbientSoilPeriod\signals\Suelo02_Ax01.txt'
builder_X = BuildPeriod(signal_path, config , display_figures=True)

signal_path = r'C:\Users\ppala\OneDrive\01. Brain\11. GitHub\AmbientSoilPeriod\signals\Suelo02_Ay01.txt'
builder_Y = BuildPeriod(signal_path, config , display_figures=True)

signal_path = r'C:\Users\ppala\OneDrive\01. Brain\11. GitHub\AmbientSoilPeriod\signals\Suelo02_Az01.txt'
builder_Z = BuildPeriod(signal_path, config , display_figures=True)


In [ ]:
aaaa=builder_X.mfx

In [ ]:
import numpy as np
from ambientperiod.analysis.prom_vent import prom_vent
from ambientperiod.tools.plot_spectrum import plot_spectrum
from ambientperiod.analysis.sua_vent import sua_vent

# --- Parámetros
Fs = config["Fs"]
vent = config["vent"]
vent_samples = int(Fs * vent)
p = config["p"]
f1 = config["f1"]
f2 = config["f2"]
bexp = config["bexp"]

# --- Paso 1: Ventanas comunes reales
pos_X = builder_X.windows_pos
pos_Y = builder_Y.windows_pos
pos_Z = builder_Z.windows_pos

# Posiciones comunes usando lógica exacta de MATLAB
pos_total_o = np.concatenate((pos_X, pos_Y))
_, idx_unique = np.unique(pos_total_o, return_index=True)
idx_duplicates = np.setdiff1d(np.arange(len(pos_total_o)), idx_unique)
pos_total_1 = np.unique(pos_total_o[idx_duplicates])

pos_total_2 = np.concatenate((pos_total_1, pos_Z))
_, idx_unique = np.unique(pos_total_2, return_index=True)
idx_duplicates = np.setdiff1d(np.arange(len(pos_total_2)), idx_unique)
pos_total = np.unique(pos_total_2[idx_duplicates])

# --- Paso 2: Cortar ventanas crudas
def cortar_ventanas(signal, pos_total):
    ventanas = []
    for pos in pos_total:
        tramo = signal[pos:pos+vent_samples]
        if len(tramo) == vent_samples:
            ventanas.append(tramo)
    return np.array(ventanas).T  # (n_samples, n_ventanas)

MV_X1 = cortar_ventanas(builder_X.signal, pos_total)
MV_Y1 = cortar_ventanas(builder_Y.signal, pos_total)
MV_Z1 = cortar_ventanas(builder_Z.signal, pos_total)

# --- Paso 3: Aplicar taper
def apply_taper(signal_matrix, p):
    n = signal_matrix.shape[0]
    taper = np.ones(n)
    np_hann = int(p * n)
    taper[:np_hann] = 0.5 * (1 - np.cos(np.pi * np.arange(np_hann) / np_hann))
    taper[-np_hann:] = 0.5 * (1 - np.cos(np.pi * np.arange(np_hann)[::-1] / np_hann))
    return signal_matrix * taper[:, None]

MA_X = apply_taper(MV_X1, p)
MA_Y = apply_taper(MV_Y1, p)
MA_Z = apply_taper(MV_Z1, p)

# --- Paso 4: FFT (solo magnitud)
def compute_fft(signal_matrix, Fs, f1, f2):
    n = signal_matrix.shape[0]
    freqs = np.fft.rfftfreq(n, d=1/Fs)
    fft_vals = np.abs(np.fft.rfft(signal_matrix, axis=0))
    # Filtrado de banda
    idx = (freqs >= f1) & (freqs <= f2)
    return freqs[idx], fft_vals[idx, :]

MFX_X, MFZ_X = compute_fft(MA_X, Fs, f1, f2)
_, MFZ_Y = compute_fft(MA_Y, Fs, f1, f2)
_, MFZ_Z = compute_fft(MA_Z, Fs, f1, f2)

# --- Paso 5: Suavizado espectral
MFS_X = sua_vent(MFX_X, MFZ_X, bexp)
MFS_Y = sua_vent(MFX_X, MFZ_Y, bexp)
MFS_Z = sua_vent(MFX_X, MFZ_Z, bexp)

# --- Paso 6: Cálculo H/V Nakamura
H_Nakamura = np.sqrt(MFS_X * MFS_Y)
H_V = H_Nakamura / MFS_Z

# --- Paso 7: Promedio
mfx = MFX_X
mfp_HV = prom_vent(H_V)

# --- Paso 8: Graficar
plot_spectrum(mfx, H_V, mfp_HV, peak_spacing_hz=0.5)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

def plot_multi_spectra(mfx_X, mfs_X, mfx_Y, mfs_Y, mfx_Z, mfs_Z, mfx_HV, HV, mfp_HV, peak_spacing_hz=0.5):
    """
    Grafica los espectros suavizados de X, Y, Z y el espectro H/V con su promedio.

    Parámetros:
    -----------
    mfx_X, mfx_Y, mfx_Z : ndarray
        Frecuencias de cada componente (X, Y, Z).
    mfs_X, mfs_Y, mfs_Z : ndarray
        Espectros suavizados de cada componente.
    mfx_HV : ndarray
        Frecuencia común para H/V.
    HV : ndarray
        Matriz de H/V Nakamura.
    mfp_HV : ndarray
        Promedio del espectro H/V.
    peak_spacing_hz : float
        Espaciado mínimo entre picos para el análisis.
    """

    fig, axs = plt.subplots(4, 1, figsize=(10, 10), sharex=False, facecolor='white')

    axs[0].semilogx(mfx_X[:, 0], mfs_X, alpha=0.6)
    axs[0].set_ylabel('Amp X', fontweight='bold')
    axs[0].set_title('Smoothed Spectrum X', fontweight='bold')
    axs[0].grid(True, which='both')

    axs[1].semilogx(mfx_Y[:, 0], mfs_Y, alpha=0.6)
    axs[1].set_ylabel('Amp Y', fontweight='bold')
    axs[1].set_title('Smoothed Spectrum Y', fontweight='bold')
    axs[1].grid(True, which='both')

    axs[2].semilogx(mfx_Z[:, 0], mfs_Z, alpha=0.6)
    axs[2].set_ylabel('Amp Z', fontweight='bold')
    axs[2].set_title('Smoothed Spectrum Z', fontweight='bold')
    axs[2].grid(True, which='both')

    axs[3].semilogx(mfx_HV[:, 0], HV, color='gray', alpha=0.4)
    axs[3].semilogx(mfx_HV[:, 0], mfp_HV, color='black', linewidth=2, label='Average H/V')
    axs[3].set_xlabel('Frequency [Hz]', fontweight='bold')
    axs[3].set_ylabel('H/V', fontweight='bold')
    axs[3].set_title('H/V Nakamura', fontweight='bold')
    axs[3].grid(True, which='both')

    # Picos
    df = mfx_HV[1, 0] - mfx_HV[0, 0]
    peak_spacing_samples = int(peak_spacing_hz / df)
    peaks, _ = find_peaks(mfp_HV, distance=peak_spacing_samples)
    sorted_peaks = peaks[np.argsort(mfp_HV[peaks])[::-1]]
    top_peaks = sorted_peaks[:4]

    for idx in top_peaks:
        f = mfx_HV[idx, 0]
        T = 1.0 / f if f != 0 else 0.0
        label = f'f={f:.2f} Hz\nT={T:.2f} s'
        axs[3].text(f, mfp_HV[idx], label, fontsize=8, ha='center', va='bottom')

    axs[3].legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# === ENCONTRAR VENTANAS COMUNES ===
pos_X = builder_X.windows_pos
pos_Y = builder_Y.windows_pos
pos_Z = builder_Z.windows_pos
# Paso 1: Ventanas comunes
pos_total = np.intersect1d(np.intersect1d(builder_X.windows_pos, builder_Y.windows_pos), builder_Z.windows_pos)

# Paso 2: Filtrar espectros suavizados por posición
def filtrar_espectros_por_posicion(pos_original, espectros, pos_comunes):
    idx_comunes = [i for i, p in enumerate(pos_original) if p in pos_comunes]
    return espectros[:, idx_comunes]

mfs_X_common = filtrar_espectros_por_posicion(builder_X.windows_pos, builder_X.mfs, pos_total)
mfs_Y_common = filtrar_espectros_por_posicion(builder_Y.windows_pos, builder_Y.mfs, pos_total)
mfs_Z_common = filtrar_espectros_por_posicion(builder_Z.windows_pos, builder_Z.mfs, pos_total)

# Paso 3: Calcular H/V ventana a ventana
HV_individual = np.sqrt(mfs_X_common * mfs_Y_common) / mfs_Z_common

# Paso 4: Promedio logarítmico
HV_log = np.log(HV_individual + 1e-12)  # evitar log(0)
HV_mean_log = np.mean(HV_log, axis=1)
H_V_final = np.exp(HV_mean_log)

# Paso 5: Plot
plot_spectrum(builder_X.mfx, HV_individual, H_V_final)
